<a href="https://colab.research.google.com/github/safal25/cv/blob/master/IMDB_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
imdb = keras.datasets.imdb

In [ ]:
vocab_size=10000
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
train_data[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [ ]:
word_index=imdb.get_word_index()

word_index={k:(v+3) for k,v in word_index.items()}

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [ ]:
s=["the","movie","was","beautiful"]
arr=[word_index[k] for k in s]

In [ ]:
arr

[4, 20, 16, 307]

In [ ]:
reverse_word_index=dict([(value,key) for key,value in word_index.items()])

In [ ]:
def decode_review(text):

  return " ".join([reverse_word_index.get(i,'?') for i in text])

review=decode_review(train_data[0])

In [ ]:
for i in review:
  print(i,end='')

<START this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what

In [ ]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [ ]:
train_data[0]

In [ ]:
model=keras.Sequential([keras.layers.Embedding(vocab_size,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 2s 32ms/step - loss: 0.6918 - accuracy: 0.5784 - val_loss: 0.6898 - val_accuracy: 0.5829
Epoch 2/30
49/49 [==============================] - 1s 29ms/step - loss: 0.6841 - accuracy: 0.6851 - val_loss: 0.6772 - val_accuracy: 0.7416
Epoch 3/30
49/49 [==============================] - 1s 29ms/step - loss: 0.6612 - accuracy: 0.7364 - val_loss: 0.6454 - val_accuracy: 0.7615
Epoch 4/30
49/49 [==============================] - 1s 29ms/step - loss: 0.6155 - accuracy: 0.7813 - val_loss: 0.5947 - val_accuracy: 0.7830
Epoch 5/30
49/49 [==============================] - 1s 29ms/step - loss: 0.5542 - accuracy: 0.8106 - val_loss: 0.5374 - val_accuracy: 0.8016
Epoch 6/30
49/49 [==============================] - 1s 29ms/step - loss: 0.4905 - accuracy: 0.8370 - val_loss: 0.4806 - val_accuracy: 0.8303
Epoch 7/30
49/49 [==============================] - 1s 29ms/step - loss: 0.4334 - accuracy: 0.8568 - val_loss: 0.4369 - val_accuracy: 0.8367
Epoch 8/30
49

In [ ]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 1s 1ms/step - loss: 0.2931 - accuracy: 0.8850


In [ ]:
print(accuracy*100,end="")
print("%")

88.50399851799011%


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
string="real talent and story comes in webseries we love it awesome storyline screenplay and best music kudos to the production team"


In [ ]:
arr=string.split()

In [ ]:
arr

['real',
 'talent',
 'and',
 'story',
 'comes',
 'in',
 'webseries',
 'we',
 'love',
 'it',
 'awesome',
 'storyline',
 'screenplay',
 'and',
 'best',
 'music',
 'kudos',
 'to',
 'the',
 'production',
 'team']

In [ ]:
def review_encoder(text):
  arr=[word_index.get(word,0) for word in text]
  return arr

scam_review=review_encoder(arr)

In [ ]:
scam_review=np.array([scam_review])

In [ ]:
scam_review.shape

(1, 21)

In [ ]:
scam_review=keras.preprocessing.sequence.pad_sequences(scam_review,value=word_index["<PAD>"],padding='post',maxlen=500)

In [ ]:
scam_review

array([[ 147,  676,    5,   65,  266,   11,    0,   75,  119,   12, 1190,
         769,  881,    5,  118,  228, 3600,    8,    4,  365,  768,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [ ]:
model.predict_classes(scam_review)

array([[1]], dtype=int32)